In this assignment, you will build unigram, bigram, and trigram character language models (both unsmoothed and smoothed versions) for three languages, score a test document with each, and determine the language it is written in based on perplexity. You will also use your English language models to generate texts. You will critically examine all results. The learning goals of this assignment are to:

- Understand how to compute language model probabilities using maximum likelihood estimation.
- Implement basic smoothing and interpolation.
- Use the perplexity of a language model to perform language identification.
- Use a language model to probabilistically generate texts.

## Data

The data for this project is available here: hw3_data.zip. It consists of:

- training.en - English training data
- training.es - Spanish training data
- training.de - German training data
- test - test document

In [141]:
import pandas as pd

filenames = ['hw3_data/training.en.txt', 'hw3_data/training.de.txt', 'hw3_data/training.es.txt', 'hw3_data/test.txt']
with open(filenames[0], 'r') as f:
    en = f.read().lower()
with open(filenames[1], 'r') as f:
    de = f.read().lower()
with open(filenames[2], 'r') as f:
    es = f.read().lower()
with open(filenames[3], 'r') as f:
    test = f.read().lower()

# en = pd.read_csv('hw3_data/training.en.txt', sep=' ', header = None)
# es = pd.read_csv()
# de = pd.read_csv()
# test = pd.read_csv()

## 1. Train character n-gram language models
To complete the assignment, you will need to write a program (from scratch) that:

- builds the models: reads in training data, collects counts for all character 1, 2, and 3-grams, estimates probabilities, and writes out the unigram, bigram, and trigram models (ngram probabilities) into files. Build separate models for each language.
- adjusts the counts: rebuilds the bigram and trigram language models using linear interpolation with lambdas equally weighted

You may make any additional assumptions and design decisions, but state them in your report (see below). For example, some design choices that could be made are how you want to handle uppercase and lowercase letters or how you want to handle digits. The choice made is up to you, we only require that you detail these decisions in your report and consider any implications of them in your results. There is no wrong choice here, and these decisions are typically made by NLP researchers when pre-processing data.

You may write your program in any TA-approved programming language (Python, Java, C/C++).

For this assignment you must implement the model generation from scratch, but you are allowed to use any resources or packages that help you manage your project, i.e. Github or any file i/o packages. If you have questions about this please ask.

Extra credit (3 points): Also implement add-one smoothing.

I make all words lower case. If there are any words that have different meanings when they are uppercase vs. lowercase, perhaps indicating a proper noun versus an ordinary noun, this will convolve the counts of two different words together, which is problematic for the representations of the affected words. I have ignored dealing with digits, so if there is 'seven' and '7', these will have separate counts and be considered separate individual words.

A little bit of preproc - adding some "\<s>" tokens

In [142]:
# en = en.replace('.\n', ' </s> <s> ').replace('\n', ' </s> <s> ').replace(',', '').replace('.', ' <s> </s> ').split()
# es = es.replace('.\n', ' </s> <s> ').replace('\n', ' </s> <s> ').replace(',', '').replace('.', ' <s> </s> ').split()
# de = de.replace('.\n', ' </s> <s> ').replace('\n', ' </s> <s> ').replace(',', '').replace('.', ' <s> </s> ').split()
# test = test.replace('.\n', ' </s> <s> ').replace('\n', ' </s> <s> ').replace(',', '').replace('.', ' <s> </s> ').split()
en = en.replace('\n', '').replace(',', '').split()
es = es.replace('\n', '').replace(',', '').split()
de = de.replace('\n', '').replace(',', '').split()
test = test.replace('\n', '').replace(',', '').split()

Unigrams

In [95]:
en_uni_counts = {}
for word in en:
    for char in word:
        if char in en_uni_counts:
            en_uni_counts[char] += 1
        else:
            en_uni_counts[char] = 1

es_uni_counts = {}
for word in es:
    for char in word:
        if char in es_uni_counts:
            es_uni_counts[char] += 1
        else:
            es_uni_counts[char] = 1

de_uni_counts = {}
for word in de:
    for char in word:
        if char in de_uni_counts:
            de_uni_counts[char] += 1
        else:
            de_uni_counts[char] = 1

Bigrams

In [96]:
import numpy as np
from scipy.sparse import lil_matrix

en_bigrams = [(en[i], en[i+1]) for i in range(len(en) - 1)]
en_bi_counts = {}
for bigram in en_bigrams:
    if bigram in en_bi_counts:
        en_bi_counts[bigram] += 1
    else:
        en_bi_counts[bigram] = 1

en_bi_matrix = lil_matrix((len(en_uni_counts), len(en_uni_counts)))
en_uni_index = {unigram: index for index, unigram in enumerate(en_uni_counts)}
for bigram, count in en_bi_counts.items():
    en_bi_matrix[en_uni_index[bigram[0]], en_uni_index[bigram[1]]] = count

es_bigrams = [(es[i], es[i+1]) for i in range(len(es) - 1)]
es_bi_counts = {}
for bigram in es_bigrams:
    if bigram in es_bi_counts:
        es_bi_counts[bigram] += 1
    else:
        es_bi_counts[bigram] = 1

es_bi_matrix = lil_matrix((len(es_uni_counts), len(es_uni_counts)))
es_uni_index = {unigram: index for index, unigram in enumerate(es_uni_counts)}
for bigram, count in es_bi_counts.items():
    es_bi_matrix[es_uni_index[bigram[0]], es_uni_index[bigram[1]]] = count

de_bigrams = [(de[i], de[i+1]) for i in range(len(de) - 1)]
de_bi_counts = {}
for bigram in de_bigrams:
    if bigram in de_bi_counts:
        de_bi_counts[bigram] += 1
    else:
        de_bi_counts[bigram] = 1

de_bi_matrix = lil_matrix((len(de_uni_counts), len(de_uni_counts)))
de_uni_index = {unigram: index for index, unigram in enumerate(de_uni_counts)}
for bigram, count in de_bi_counts.items():
    de_bi_matrix[de_uni_index[bigram[0]], de_uni_index[bigram[1]]] = count

Trigrams

In [97]:
en_trigrams = [(en[i], en[i+1], en[i+2]) for i in range(len(en) - 2)]
en_tri_counts = {}
for trigram in en_trigrams:
    if trigram in en_tri_counts:
        en_tri_counts[trigram] += 1
    else:
        en_tri_counts[trigram] = 1

en_tri_matrix = lil_matrix((len(en_bi_counts), len(en_uni_counts)))
en_bi_index = {bigram: index for index, bigram in enumerate(en_bi_counts)}
for trigram, count in en_tri_counts.items():
    en_tri_matrix[en_bi_index[trigram[0:2]], en_uni_index[trigram[2]]] = count

es_trigrams = [(es[i], es[i+1], es[i+2]) for i in range(len(es) - 2)]
es_tri_counts = {}
for trigram in es_trigrams:
    if trigram in es_tri_counts:
        es_tri_counts[trigram] += 1
    else:
        es_tri_counts[trigram] = 1

es_tri_matrix = lil_matrix((len(es_bi_counts), len(es_uni_counts)))
es_bi_index = {bigram: index for index, bigram in enumerate(es_bi_counts)}
for trigram, count in es_tri_counts.items():
    es_tri_matrix[es_bi_index[trigram[0:2]], es_uni_index[trigram[2]]] = count

de_trigrams = [(de[i], de[i+1], de[i+2]) for i in range(len(de) - 2)]
de_tri_counts = {}
for trigram in de_trigrams:
    if trigram in de_tri_counts:
        de_tri_counts[trigram] += 1
    else:
        de_tri_counts[trigram] = 1

de_tri_matrix = lil_matrix((len(de_bi_counts), len(de_uni_counts)))
de_bi_index = {bigram: index for index, bigram in enumerate(de_bi_counts)}
for trigram, count in de_tri_counts.items():
    de_tri_matrix[de_bi_index[trigram[0:2]], de_uni_index[trigram[2]]] = count

"writes out the unigram, bigram, and trigram models (ngram probabilities) into files. Build separate models for each language."

Convert ngrams into probabilities and write to files

In [124]:
en_total = sum(en_uni_counts.values())
es_total = sum(es_uni_counts.values())
de_total = sum(de_uni_counts.values())

en_uni_probs = {key: value / en_total for key, value in en_uni_counts.items()}
es_uni_probs = {key: value / es_total for key, value in es_uni_counts.items()}
de_uni_probs = {key: value / de_total for key, value in de_uni_counts.items()}

In [125]:
sum(en_uni_probs.values())

1.0000000000000326

The bigram matrix has 3403 unique words. If the probability of each word, given all its contexts is 1, then the sum of the column sums should be 3403.

In [129]:
print(en_bi_matrix.shape, es_bi_matrix.shape, de_bi_matrix.shape)

(3403, 3403) (4332, 4332) (4412, 4412)


In [118]:
en_bi_sums = en_bi_matrix.sum(axis = 0) # sum rows of each column
en_bi_sums[en_bi_sums == 0] = 1 # no div by 0
en_bi_matrix_norm = en_bi_matrix / en_bi_sums

es_bi_sums = es_bi_matrix.sum(axis = 0) # sum over rows of each column
es_bi_sums[es_bi_sums == 0] = 1 # no div by 0
es_bi_matrix_norm = es_bi_matrix / es_bi_sums

de_bi_sums = de_bi_matrix.sum(axis = 0) # sum rows of each column
de_bi_sums[de_bi_sums == 0] = 1 # no div by 0
de_bi_matrix_norm = de_bi_matrix / de_bi_sums

In [131]:
print(sum(en_bi_matrix_norm.sum(axis=1)), sum(es_bi_matrix_norm.sum(axis=1)), sum(de_bi_matrix_norm.sum(axis=1)))

[[3402.]] [[4331.]] [[4412.]]


"Resumption" is a word in the vocabulary which does not follow any word, thus has a probability of zero, that the sum is 3402 is indication that the probabilities for each word summed across contexts is one. The same logic applies to the trigrams.

In [130]:
print(en_tri_matrix.shape, es_tri_matrix.shape, de_tri_matrix.shape)

(15078, 3403) (15508, 4332) (15668, 4412)


In [126]:
en_tri_sums = en_tri_matrix.sum(axis = 0) # sum rows of each column
en_tri_sums[en_tri_sums == 0] = 1 # no div by 0
en_tri_matrix_norm = en_tri_matrix / en_tri_sums

es_tri_sums = es_tri_matrix.sum(axis = 0) # sum over rows of each column
es_tri_sums[es_tri_sums == 0] = 1 # no div by 0
es_tri_matrix_norm = es_tri_matrix / es_tri_sums

de_tri_sums = de_tri_matrix.sum(axis = 0) # sum rows of each column
de_tri_sums[de_tri_sums == 0] = 1 # no div by 0
de_tri_matrix_norm = de_tri_matrix / de_tri_sums

In [132]:
print(sum(en_tri_matrix_norm.sum(axis=1)), sum(es_tri_matrix_norm.sum(axis=1)), sum(de_tri_matrix_norm.sum(axis=1)))

[[3402.]] [[4331.]] [[4412.]]
